## Scrapping reviews for courses

In [ ]:
import csv,requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import multiprocessing as mp
chromeOptions = webdriver.ChromeOptions()
chromeOptions.add_argument("--remote-debugging-port=9222")

In [ ]:
subject_list = ["Math and Logic" , "Personal Development","Arts and Humanities" ,"Social Sciences","Language Learning","Health"]


In [ ]:
def scrapper(obj):
    ob = obj[1]
    review_df = pd.DataFrame()
    driver = webdriver.Chrome(ChromeDriverManager().install())
    course = driver.get("{}".format(ob["course_link"]) + "/reviews")
    try:
        count = int(driver.find_element(By.XPATH,"//a[contains(@data,'right-arrow')]/parent::li/preceding-sibling::li[1]/a/span").text)
    except:
        count = 1
    print(count)
    while count > 0:
        count -= 1
        try:
            review_list = driver.find_elements(By.XPATH, "//div[contains(@class,'review-text')]")
            for review in review_list:
                soup = BeautifulSoup(review.get_attribute('innerHTML'), 'html.parser')
                review_star = soup.find_all('title', string="Filled Star")
                review_text = soup.find('div',class_="reviewText").text
                review_df = review_df.append({"star":len(review_star),"review":review_text,"course_id":ob["course_id"]},ignore_index=True)
            if count > 1:
                driver.find_element(By.XPATH, "//a[contains(@data,'right-arrow')]").click()
                wait = WebDriverWait(driver, 7)
                element = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@data,'right-arrow')]")))
            print("review: ",review_df.shape[0])
        except Exception as e:
            print("Exception")
            print(ob["course_id"])
            print(count)
            print(e)
    print("end")
    review_df.to_csv(r"Final_reviews.csv", index=False,mode='a', header=False)


In [ ]:
pool = mp.Pool(5)
for subject in subject_list:
    csv_file = pd.read_csv(r"course_link\{}.csv".format(subject))
    pool.map(scrapper, list(csv_file.iterrows()))